In [30]:
import gym
import d4rl

import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('../')

import torch
import torch.nn as nn
from torch import optim
import copy


from torchvision import transforms, utils, datasets
from torch.utils.data import  DataLoader, Dataset
import torchvision.utils as vutils
import torch.nn.functional as F

from models import DQN_fc

In [31]:
no_cuda = True
if torch.cuda.is_available() and not no_cuda:
    device = torch.device("cuda")
    torch.cuda.set_device(0)
else:
    device = torch.device("cpu")

In [32]:
print(device)

cpu


In [77]:
# env_name = 'maze2d-medium-v1'
env_name = 'CartPole-v0'
env = gym.make(env_name)
env = gym.make(env_name)
is_atari = gym.envs.registry.spec(env_name).entry_point == 'gym.envs.atari:AtariEnv'

print(is_atari)

False


In [83]:
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
# max_action = float(env.action_space.)

print('state dim:{}'.format(state_dim))
print('action dim:{}'.format(action_dim))
print('state shape:{}'.format(env.observation_space.shape))




state dim:4
action dim:2
state shape:(4,)


### Replay buffer


In [84]:
class ReplayBuffer(object):
    def __init__(self, state_dim, action_dim, batch_size, buffer_size, device):
        self.batch_size = batch_size
        self.max_size = int(buffer_size)
        self.device = device

        self.ptr = 0
        self.crt_size = 0

        self.state = np.zeros((self.max_size, state_dim))
        self.action = np.zeros((self.max_size, action_dim))
        self.next_state = np.array(self.state)
        self.reward = np.zeros((self.max_size, 1))
        self.not_done = np.zeros((self.max_size, 1))


    def add(self, state, action, next_state, reward, done):
        self.state[self.ptr] = state
        self.action[self.ptr] = action
        self.next_state[self.ptr] = next_state
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.crt_size = min(self.crt_size + 1, self.max_size)


    def sample(self):
        ind = np.random.randint(0, self.crt_size, size=self.batch_size)
        return (
            torch.FloatTensor(self.state[ind, :]).to(self.device),
            torch.LongTensor(self.action[ind]).to(self.device),
            torch.FloatTensor(self.next_state[ind, :]).to(self.device),
            torch.FloatTensor(self.reward[ind]).to(self.device),
            torch.FloatTensor(self.not_done[ind]).to(self.device)
        )


    def save(self, save_folder):
        np.save(f"{save_folder}_state.npy", self.state[:self.crt_size])
        np.save(f"{save_folder}_action.npy", self.action[:self.crt_size])
        np.save(f"{save_folder}_next_state.npy", self.next_state[:self.crt_size])
        np.save(f"{save_folder}_reward.npy", self.reward[:self.crt_size])
        np.save(f"{save_folder}_not_done.npy", self.not_done[:self.crt_size])
        np.save(f"{save_folder}_ptr.npy", self.ptr)


    def load(self, save_folder, size=-1):
        reward_buffer = np.load(f"{save_folder}_reward.npy")

        # Adjust crt_size if we're using a custom size
        size = min(int(size), self.max_size) if size > 0 else self.max_size
        self.crt_size = min(reward_buffer.shape[0], size)

        self.state[:self.crt_size] = np.load(f"{save_folder}_state.npy")[:self.crt_size]
        self.action[:self.crt_size] = np.load(f"{save_folder}_action.npy")[:self.crt_size]
        self.next_state[:self.crt_size] = np.load(f"{save_folder}_next_state.npy")[:self.crt_size]
        self.reward[:self.crt_size] = reward_buffer[:self.crt_size]
        self.not_done[:self.crt_size] = np.load(f"{save_folder}_not_done.npy")[:self.crt_size]

        print(f"Replay Buffer loaded with {self.crt_size} elements.")

In [85]:
buffer_size = 1e6
batch_size = 64
replay_buffer = ReplayBuffer(state_dim, action_dim, batch_size, buffer_size, device)

### Dataset preparation

In [86]:
# add dataset to the replay buffer

# dataset = env.get_dataset()
dataset = d4rl.qlearning_dataset(env)

N = dataset['rewards'].shape[0]
print('Loading buffer!')
for i in range(N):
    obs = dataset['observations'][i]
    new_obs = dataset['observations'][i]
    action = dataset['actions'][i]
    reward = dataset['rewards'][i]
    done_bool = bool(dataset['terminals'][i])
    replay_buffer.add(obs, action, new_obs, reward, done_bool)
print('Loaded buffer')
replay_buffer.save('../buffers/{}'.format(env_name))

AttributeError: 'CartPoleEnv' object has no attribute 'get_dataset'

In [9]:
replay_buffer.load('../buffers/{}'.format(env_name))

Replay Buffer loaded with 1000000 elements.


In [10]:
s1, s2, a ,r ,d = replay_buffer.sample()

ind:[628854 826410 810265 431175 332232 793047 230212 685823 899165 748961
 495043  99054 631813 477215  18884 142498 404209 998740 904306 749428
 125700 202121 712345 976871 177127 100216 593436 959705 219187  55585
 411418 330868 325253 640326 722373 563043 359954 889642 201665 314726
 283374 534941 709065 240131 419248 881084 516390 680063 754575 105060
 453075 596235 338575 803498 284739 817068 671859 431516 555516 203580
 562115 470280   8591  26205]


In [11]:
print(s1.size())

torch.Size([64, 4])


### models and agent

In [72]:
class discrete_BCQ(object):
    def __init__(self, action_dim, state_dim, device, discount=0.99, optimizer= "Adam", 
                 optimizer_parameters={}, target_update_frequency=8e3, initial_eps = 1, 
                 end_eps = 0.001, eps_decay_period = 25e4, eval_eps=0.001):
        self.device = device
        self.Q = DQN_fc(state_dim, action_dim)
        self.Q_target = copy.deepcopy(self.Q)
        self.Q_optimizer = getattr(torch.optim, optimizer)(self.Q.parameters(), **optimizer_parameters)
        self.Q = self.Q.to(device)
        self.Q_target = self.Q_target.to(device)

        self.discount = discount
        self.target_update_frequency = target_update_frequency

            # Decay for eps
        self.initial_eps = initial_eps
        self.end_eps = end_eps
        self.slope = (self.end_eps - self.initial_eps) / eps_decay_period

        # Evaluation hyper-parameters
        self.state_shape = (-1, state_dim) ### need to pass framesize
        self.eval_eps = eval_eps
        self.action_dim = action_dim

        # Number of training iterations
        self.iterations = 0
    
    def select_action(self, state, eval=False):
        # eps for eval is fixed 
        if eval:
            eps = self.eval_eps
        else:
            eps = max(self.slope * self.iterations + self.initial_eps, self.end_eps)
            
        # Select action according to policy with probability (1-eps)
        # otherwise, select random action
        if np.random.uniform(0, 1) > eps:
            with torch.no_grad():
                state = torch.FloatTensor(state).reshape(state_shape).to(device)
                return int(self.Q(state).argmax(1))
        else:
            return np.random.randint(self.num_actions)

    def train(self, replay_buffer):
        
        # 1- Sample replay buffer
        state, action, next_state, reward, done = replay_buffer.sample()

        # 2- Compute the target Q value
        with torch.no_grad():
            target_Q = reward + done * self.discount * self.Q_target(next_state).max(1, keepdim=True)[0]

        # Get current Q estimate
        print(action[:5])
        print('target_Q:{}'.format(target_Q[:5]))
        current_Q = self.Q(state)

        print('current_Q:{}'.format(current_Q[:5]))
        current_Q = self.Q(state).gather(1, action)

        




        # Compute Q loss
        Q_loss = F.smooth_l1_loss(current_Q, target_Q)

        # Optimize the Q
        self.Q_optimizer.zero_grad()
        Q_loss.backward()
        self.Q_optimizer.step()

        # Update target network by polyak or full copy every X iterations.
        self.iterations += 1
        if self.iterations % self.target_update_frequency == 0:
            self.Q_target.load_state_dict(self.Q.state_dict())
        
    def save(self, filename):
        torch.save(self.Q.state_dict(), filename + "_Q")
        torch.save(self.Q_optimizer.state_dict(), filename + "_optimizer")

    def load(self, filename):
        self.Q.load_state_dict(torch.load(filename + "_Q"))
        self.Q_target = copy.deepcopy(self.Q)
        self.Q_optimizer.load_state_dict(torch.load(filename + "_optimizer"))
                

In [73]:
def eval_policy(policy, env_name, seed, eval_episodes=10):
    eval_env = gym.make(env_name)
    eval_env.seed(seed + 100)

    avg_reward = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        while not done:
            action = policy.select_action(np.array(state))
            state, reward, done, _ = eval_env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f}")
    print("---------------------------------------")
    return avg_reward


### main loop


In [74]:
# parametres
eval_freq = 5e3
max_timesteps = 1e6

# agen parametrs
discount = 0.99
target_update_frequency = 1
initial_eps = 0.1
end_eps = 0.1
eps_decay_period = 1
eval_eps = 0

#


In [75]:
evaluations = []
episode_num = 0
done = True
training_iters = 0

policy = discrete_BCQ(action_dim, state_dim, device, discount, "Adam", {"lr": 3e-4}, target_update_frequency,
                      initial_eps, end_eps, eps_decay_period, eval_eps)
    

while training_iters < max_timesteps:
    for _ in range(int(eval_freq)):
        policy.train(replay_buffer)


    evaluations.append(eval_policy(policy, env_name, seed))
#     np.save(os.path.join(output_dir, f"BCQ_{setting}"), evaluations)

    training_iters += int(eval_freq)
    print(f"Training iterations: {training_iters}")

tensor([[ 1,  0],
        [ 0, -1],
        [-1,  1],
        [ 0,  0],
        [-1,  0]])
target_Q:tensor([[ 0.0953],
        [-0.0360],
        [-0.0560],
        [-0.0049],
        [ 0.0390]])
current_Q:tensor([[ 0.0962, -0.0080],
        [-0.0736, -0.0363],
        [-0.0566, -0.0971],
        [-0.2861, -0.0050],
        [ 0.0394, -0.0196]], grad_fn=<SliceBackward>)


RuntimeError: index -1 is out of bounds for dimension 1 with size 2